In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from googlesearch import search
from lxml import html
import requests
from bs4 import BeautifulSoup

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from keras.layers.advanced_activations import LeakyReLU, PReLU

from sklearn.model_selection import train_test_split 
from sklearn import metrics

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/Users/codyhartsook/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv('ks-projects-201801.csv')

In [3]:
# only use a subset of our whole dataset
length = len(data['goal'])
subset = length - 10000 # if we only want 20,000 rows

data = data.iloc[0:length-subset]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
ID                  10000 non-null int64
name                10000 non-null object
category            10000 non-null object
main_category       10000 non-null object
currency            10000 non-null object
deadline            10000 non-null object
goal                10000 non-null float64
launched            10000 non-null object
pledged             10000 non-null float64
state               10000 non-null object
backers             10000 non-null int64
country             10000 non-null object
usd pledged         9898 non-null float64
usd_pledged_real    10000 non-null float64
usd_goal_real       10000 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 1.1+ MB


In [12]:
live_data = data.loc[data['state'] == 'live']
live_data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
40,1000149007,Unschooling To University Book Project,Nonfiction,Publishing,CAD,2018-01-19,3000.0,2017-11-20 18:15:14,592.0,live,24,CA,185.65,472.88,2396.36
183,1000761521,KILOS.,Thrillers,Film & Video,USD,2018-02-06,5000.0,2017-12-23 20:34:31,310.0,live,4,US,200.00,310.00,5000.00
458,1002183790,The Man in the Field,Horror,Film & Video,GBP,2018-01-08,850.0,2017-12-21 11:19:18,413.0,live,11,GB,13.39,563.32,1159.38
537,1002599057,Dear Atlantas,Film & Video,Film & Video,USD,2018-02-16,2000.0,2017-12-18 02:48:27,5.0,live,1,US,5.00,5.00,2000.00
544,1002629894,Art Calendar 2018,Illustration,Art,MXN,2018-01-06,1000.0,2017-12-21 05:34:12,0.0,live,0,MX,0.00,0.00,52.32


In [ ]:
def scrape(url):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    header = soup.find_all('h3')
    if len(header) > 0:
    
        rewards = []
        count = 0
        vid = 0
        for elem in header:
            line = str(elem)
            
            if line.find("pledge__title") >= 0:
                reward_obj = line.split("\n")
                #rewards.append(reward_obj[1])
                count += 1
            elif line.find("</video>") >= 0:
                vid += 1
        return count, vid
    else:
        return 0, 0

In [ ]:
itr = 0
reward_num = []
vid_num = []
to_delete = []

for name in data['name']:
    query = "www.kickstarter.com/projects/" + str(name)

    valid  = False
    reward = 0
    vid = 0
    
    if itr % 4 == 0:
        delay = random.randint(3, 8)
        time.sleep(delay)
    
    delay = 1
    time.sleep(delay)
    
    try:
        failed = 0
        for j in search(query, tld="co.in", num=2, stop=1, pause=1): 
            if str(j).find("www.kickstarter.com") >= 0:
                valid = True

            reward, video = scrape(j)
                if reward > 0:
                    break
        if valid == False:
            # delete itr row from dataset since it was not found on kickstarter
            to_delete.append(itr)
            itr += 1
            continue
    
    except:
        print("failed")
        # delay the process
        delay = 150
        time.sleep(delay)
        print("continue")
        
        failed = 1
        to_delete.append(itr)
        
    print(reward)
    print(vid)
    reward_num.append(reward)
    vid_num.append(vid)
    itr += 1
    
data['pledge_rewards'] = reward_num
data['vid_num'] = vid_num
for row in to_delete:
    data = data.drop(data.index[row])

In [ ]:
# number of days between launch data and deadline
def duration_of_project():
    duration = []
    
    for launch, deadline in zip(data['launched'], data['deadline']):
        
        launch = (launch.split())[0]
        launchT = [x.strip() for x in launch.split('-')]
        deadlineT = [x.strip() for x in deadline.split('-')]
    
        year_diff = (int(deadlineT[0]) - int(launchT[0]))*365
        month_diff = (int(deadlineT[1]) - int(launchT[1]))*30
        day_diff = int(deadlineT[2]) - int(launchT[2])
    
        days_offset = year_diff + month_diff + day_diff
        duration.append(days_offset)
        
    data['duration'] = duration

In [ ]:
duration_of_project()

In [ ]:
#add_scraped()

In [ ]:
#getting unique categories in a list
cat = []
for i in data['category']:
    if i not in cat:
        cat.append(i)
#creating the unique categories successful mean
print(len(cat))
BackerMean = [0] * (len(cat) + 1)
GoalMean = [0] * (len(cat) + 1)

counter = 0
for i in cat:
    count = 0
    total = 0
    for j in data['category']:
        if j==cat[counter]:
            if data['state'][count] == 'successful':
                BackerMean[counter] += data['backers'][count]
                GoalMean[counter] += data['goal'][count]
                total = total + 1
        count = count + 1
    if BackerMean[counter]!= 0:
        BackerMean[counter] = BackerMean[counter]/total
        GoalMean[counter] = GoalMean[counter]/total
    BackerMean[counter] = round(BackerMean[counter])
    GoalMean[counter] = round(GoalMean[counter])
    counter = counter + 1

In [ ]:
#Creates a Column stating a binary state on whether or not the project had reached the mean
#of backers in its prospective category
length = len(data['category'])
BackerBinary = [0] * length
CategoryMean = [0] * length
GoalBinary = [0] * length
counter = 0
for i in cat:
    count = 0
    for j in data['category']:
        if j == i:
            BackerBinary[count] = data['backers'][count] - BackerMean[counter]
            GoalBinary[count] = data['goal'][count] - GoalMean[counter]
            CategoryMean[count] = BackerMean[counter]
            if BackerBinary[count] < 0:
                BackerBinary[count] = 0
            else:
                BackerBinary[count] = 1
            if GoalBinary[count] > 0:
              GoalBinary[count] = 0
            else:
              GoalBinary[count] = 1
        count = count+1
    counter = counter + 1

In [ ]:
data['GoalBinary'] = GoalBinary
data['state'].replace(['canceled'], ['failed'], inplace=True)
data['state'].replace(['failed', 'successful'], [-1, 1], inplace=True)

In [ ]:
data['state_binary'] = data['state']
data['state_binary'].replace([-1, 1], [0, 1], inplace=True)

In [ ]:
slim_data = data.copy()
indexNames = slim_data[ (slim_data['state_binary'] != 1) & (slim_data['state_binary'] != 0) ].index
slim_data.drop(indexNames , inplace=True)

In [ ]:
# find length of name
name_length = []
for name in slim_data['name']:
    try:
        name_length.append(len(name))
    except:
        name_length.append(0)
slim_data['name_len'] = name_length

In [ ]:
# scale down data columns
sc = MinMaxScaler()

slim_data['fit_goal'] = sc.fit_transform(slim_data[['goal']])
slim_data['fit_name_len'] = sc.fit_transform(slim_data[['name_len']])
slim_data['fit_duration'] = sc.fit_transform(slim_data[['duration']])

In [ ]:
training=pd.get_dummies(slim_data, columns=["main_category"])
training=pd.get_dummies(training, columns=["category"])
training=pd.get_dummies(training, columns=["country"])
training=pd.get_dummies(training, columns=["currency"])

y = training['state_binary']
X = training.loc[:, 'fit_goal':]
X['GoalBinary'] = slim_data['GoalBinary']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
X.head()

In [ ]:
# build DNN

model = Sequential()  # sequential layered network
dims = len(X.columns) # dimension of the input data

# input layer
model.add(Dense(100, activation='linear', input_dim = dims))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))

# second layer
model.add(Dense(100, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))


# third layer
model.add(Dense(100, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))


# fourth layer
model.add(Dense(85, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))


# fith layer
model.add(Dense(75, activation='linear'))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))

# output layer
model.add(Dense(1, activation='sigmoid'))

# compile the model using rmsprop opt and mse loss
model.compile(optimizer = 'adam',  
              loss = 'mse',
              metrics = ['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
history = model.fit(X_train, y_train, 
                    validation_split = 0.33, 
                    epochs = 50,
                    batch_size = 35)

# Evaluate the model's performance
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Training set accuracy:', train_acc)
print('Test set accuracy:', test_acc)

In [ ]:
# The history of our accuracy during training.
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# The history of our cross-entropy loss during training.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(X_test)
y_pred = []
neg = pos = 0
for pred in predictions:
    if pred < 0.5:
        neg += 1
        y_pred.append(0.0)
    else:
        pos += 1
        y_pred.append(1.0)
print("pos:", pos, " | neg:", neg)
print(len(X_test))

In [ ]:
y_true = []
for val in y_test:
    y_true.append(val)
matrix = confusion_matrix(y_true, y_pred)
print(matrix)